In [ ]:
from environment import environment
import random,sys,gc,warnings
from IPython.display import clear_output

import gymnasium 
from puzzle import easyBoard

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm
torch.autograd.set_detect_anomaly(True)
warnings.filterwarnings("ignore")

In [ ]:
# hypers
batchSize = 10
lr = 1e-4
env = gymnasium.make("sudoku")

In [ ]:
class mask: # altering softmax output so x and y = {0,8} and value = {1,9}
  def __init__(self):
    self.newValue = -float("inf")

  def apply(self,tensor : torch.FloatTensor):
    self.mask = torch.zeros_like(tensor,dtype=torch.bool)
    self.mask[0,-1] = True
    self.mask[1,-1] = True
    self.mask[-1,0] = True
    tensor = tensor.masked_fill(mask=self.mask,value=self.newValue)
    return tensor
 
class network(nn.Module):
  def __init__(self):
    super().__init__()

    self.representation = nn.Sequential(
      nn.LazyConv2d(16,3,1,0),
       nn.ReLU(),
       nn.LazyConv2d(16,3,1,0),
       nn.ReLU() # torch.Size([32, 5, 5])
    )
    self.prediction = nn.Sequential(
      nn.Conv2d(16,1,1,1),
      nn.ReLU(),
      nn.Flatten(1),
      nn.LazyLinear(27),
      nn.ReLU() # torch.Size([1, 25])
    )
    self.mask = mask() # for the softmax distribution mask 
    self.policy = nn.LazyLinear(27) 
    self.value = nn.LazyLinear(1)

    self.dynamic = nn.Sequential(
      nn.LazyConv2d(32,1,1,0),
      nn.ReLU(),
      nn.LazyConv2d(32,1,1,0)
    )
    self.next_hidden_state_layer = nn.LazyConv2d(32,1,1,0)
    self.reward = nn.LazyLinear(1)

    self.optim = torch.optim.Adam(self.parameters(),lr=lr)

  def forward(self,x):
    x = F.one_hot(x.to(torch.int64),num_classes=10).to(torch.float32)
    x = self.representation(x)
    x = self.prediction(x)

    policy = self.policy(x)
    softmax = F.softmax(self.mask.apply(policy.view(3,9)))  
    value = self.value(x)
    return softmax,value

  def forward_static(self,hidden_state,action):
    x = self.dynamic(hidden_state + action) # not a real implementation...
    next_hidden_state = self.next_hidden_state_layer(x)
    reward = self.reward(x)
    return next_hidden_state,reward


model = network()
w,r = model.forward(easyBoard)

Categorical(w).sample()

In [ ]:
class collector:
    def __init__(self):
        pass
        
    def rollout(self):
        pass

    def sample(self):
        pass

    def clear_memory():
        pass

In [ ]:
def multiply(n=0,m=2,max=10,count = 0):
    if count!= max:
        print(f"{n} X {m} = {n*m}")
        multiply(n=n+1,m=2,max=10,count=count+1)
    
multiply()